In [103]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from datetime import datetime

# Load CSV data
# def load_data(data_folder):
#     dataframes = []
#     for filename in os.listdir(data_folder):
#         if filename.endswith(".csv"):
#             filepath = os.path.join(data_folder, filename)
#             df = pd.read_csv(filepath)
#             dataframes.append(df)
#     return pd.concat(dataframes)

# Example folder path
data_folder = '../NASAResources/space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1970-01-19HR00_evid00002.csv'
# Load the catalog (starting points of quakes)
catalog_path = '../NASAResources/space_apps_2024_seismic_detection/data/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv'
catalog = pd.read_csv(catalog_path)

# Load training data
#data = load_data(data_folder)


# Delete this
data = pd.read_csv(data_folder)

In [104]:

# #import multiprocessing, very slow as it uses one core

# def label_data(data, catalog):
#     labels = []
#     for index, row in data.iterrows():
#         label = 0  # Default: not a quake start
#         for _, quake in catalog.iterrows():
#             if row['time_rel'] == quake['time_rel']:  # Check if time_rel matches quake start
#                 label = 1
#                 break
#         labels.append(label)
#     return np.array(labels)

# features = data[['time_rel', 'velocity']]
# labels = label_data(data, catalog)

In [109]:


# Function to reformat the absolute time by setting the fractional seconds to .00000
def reformat_abs_time(time_str):
    dt = datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%S.%f')
    reformatted_time = dt.strftime('%Y-%m-%dT%H:%M:00') + '.000000'
    return reformatted_time


# Apply the reformatting function to the 'time_abs' column
data['time_abs'] = data['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].apply(reformat_abs_time)

# Processing data folder
data['time_rel'] = data['time_rel(sec)'].astype(float)
data['velocity'] = data['velocity(m/s)']

features = data[['time_rel', 'velocity', 'time_abs']]

catalog['time_rel'] = catalog['time_rel(sec)'].astype(float)
catalog['time_abs'] = catalog['time_abs(%Y-%m-%dT%H:%M:%S.%f)']





In [106]:
# Print the first absolute time found in the data DataFrame
print(catalog['time_abs'].iloc[12])

1970-12-11T07:22:00.000000


In [110]:

# def label_data(data, catalog):
#     # Extract absolute times from catalog and data
#     quake_start_times_abs = catalog['time_abs'].to_numpy()
#     data_times_abs = data['time_abs'].to_numpy()
    
#     # Initial check on absolute times
#     abs_labels = np.isin(data_times_abs, quake_start_times_abs).astype(int)
    
#     # Extract relative times from catalog and data
#     quake_start_times_rel = catalog['time_rel'].to_numpy()
#     data_times_rel = data['time_rel'].to_numpy()
    
#     # Second check on relative times for rows where absolute time matches
#     rel_labels = np.zeros_like(abs_labels)
#     matching_indices = np.where(abs_labels == 1)[0]
#     rel_labels[matching_indices] = np.isin(data_times_rel[matching_indices], quake_start_times_rel).astype(int)
    
#     # Combine results: only keep labels where both checks are true
#     final_labels = abs_labels & rel_labels
    
#     return final_labels

# labels = label_data(data, catalog)
# print(labels)


def label_data(data, catalog):
    # Convert catalog time_rel to a numpy array for fast lookups
    quake_start_times = catalog['time_rel'].to_numpy()
    
    # Create an array of labels
    labels = np.isin(data['time_rel'].to_numpy(), quake_start_times).astype(int)
    return labels

    # Apply labeling
    labels = label_data(data, catalog)



In [111]:
quake_start_count = np.sum(labels)
print(quake_start_count)

0
